In [38]:
from hatchet import *
import networkx as nx
import pandas
from hatchet import *
import os
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import igraph
import platform
import json
import re
from ast import literal_eval as make_tuple

In [39]:
from state import State

In [40]:
# dirname = "/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow"
dirname = "/home/vidi/Work/llnl/CallFlow/.callflow"

In [41]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [42]:
def read_gf(name):
    state = State()
    df_filepath = dirname + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
    graph_filepath = dirname + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   

    with open(graph_filepath, 'r') as graphFile:
        data = json.load(graphFile)

    state.gf = GraphFrame()
    state.gf.from_literal(data)

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
            
    state.entire_gf = GraphFrame()
    state.entire_gf.from_literal(entire_data)

    state.df = pd.read_csv(df_filepath)
    state.entire_df = pd.read_csv(entire_df_filepath)

    state.graph = state.gf.graph
    state.entire_graph = state.entire_gf.graph

    state.map = state.node_hash_mapper()

    # Print the module group by information. 
    # print(state.df.groupby(['module']).agg(['mean','count']))

    # replace df['node'] from str to the Node object.
    state.df = replace_str_with_Node(state.df, state.graph)
    state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)
    
    group_graph_filepath = dirname + '/' + name + '/group_df.csv'
    state.gdf = pd.read_csv(group_graph_filepath)
    return state

In [43]:
datasets = ["calc-pi"]
states = {}
for idx, dataset_name in enumerate(datasets):
    state = read_gf(dataset_name)
    states[dataset_name] = state

In [57]:
class groupBy:
    def __init__(self, state, group_by):
        self.state = state
        self.df = self.state.df
        self.group_by = group_by
#         self.eliminate_funcs = ['libmonitor.so.0.0.0']
        self.eliminate_funcs = []
        self.entry_funcs = {}
        self.module_func_map = {}
        self.other_funcs = {}
        self.module_id_map = {}

        self.drop_eliminate_funcs()
        self.run()
        self.df = self.state.df
        self.graph = self.state.graph
        
    def lookup_with_node(self, node):
        return self.df.loc[self.df['name'] == node.callpath[-1]]
    
    def lookup_with_name(self, name):
        return self.df.loc[self.df['name'] == name]
        
    # Drop all entries user does not want to see. 
    def drop_eliminate_funcs(self):
        for idx, func in enumerate(self.eliminate_funcs):
            self.state.df = self.state.df[self.state.df['module'] != func]

    # Create a group path for the df.column = group_path.
    def create_group_path(self, path):
        group_path = []
        self.prev_module_map = {}
        prev_module = None
        function = path[-1]        
        change_name = False

        # Create a map having initial funcs being mapped.
        module_df = self.df.groupby(['module'])
        for module, df in module_df:
            if module not in self.module_func_map:
                self.module_func_map[module] = []
            if module not in self.entry_funcs:
                self.entry_funcs[module] = []
            if module not in self.other_funcs:
                self.other_funcs[module] = []
        
        for i, elem in enumerate(path):
            grouping = self.lookup_with_name(elem)[self.group_by].unique()
            if len(grouping) == 0:
                break
            
            module = grouping[0]
                        
            # Append the module into the group path. 
            if module not in self.eliminate_funcs:
                if prev_module is None:
                    prev_module = module
                    group_path.append(module)
                elif module != prev_module:
                    # print("Group path is ", group_path)
                    if module in group_path:
                        from_module = group_path[len(group_path) - 1]
                        to_module = module
                        group_path.append(module + ':' + path[i])
                        prev_module = module
                        self.module_func_map[module].append(module + ':' + path[i])
                        change_name = True
                    else:
                        group_path.append(module)
                        prev_module = module
                        if path[i] not in self.entry_funcs[module]:
                            self.entry_funcs[module].append(path[i])
                else:
                    prev_module = module
                    continue
                    if path[i] not in self.other_funcs[module] and path[i] not in self.entry_funcs[module]:
                        self.other_funcs[module].append(path[i])
        
        group_path = tuple(group_path)
        return (group_path, change_name)

    def create_component_path(self, path, group_path):
        component_path = []
        path = list(path)
        # component_module = self.state.lookup_with_name(path[-1])[self.group_by].tolist()[0]
        component_module = group_path[-1]
        component_path = [node for node in path if component_module == \
                       self.lookup_with_name(node)[self.group_by].tolist()[0]]
        
        if len(component_path) == 0:
            component_path.append(path[-1])
        component_path.insert(0, component_module)
        return tuple(component_path)
            
    def run(self):
        group_path = {}
        component_path = {}
        component_level = {}
        entry_func = {}
        show_node = {}
        node_name = {}    
        module = {}   
        change_name = {}
        module_idx = {}

        module_id_map = {}
        module_count = 0
    
        roots = self.state.graph.roots
        if len(roots) > 1:
                ('It is a multi-rooted tree with {0} roots'.format(len(roots)))
        
        for root in roots:
            node_gen = root.traverse()       
            rootdf = self.lookup_with_name(root.callpath[-1])
            
            if rootdf.empty:
                utils.debug('Not accounting the function: {0}'.format(root))
            # Check if the dataframe exists for the root node. 
            # It might be a function that is eliminated. 
            else: 
                temp_group_path_results = self.create_group_path(root.callpath)
                group_path[rootdf.node[0]] = temp_group_path_results[0]
                change_name[rootdf.node[0]] = temp_group_path_results[1]

                component_path[rootdf.node[0]] = self.create_component_path(root.callpath, group_path[rootdf.node[0]])
                component_level[rootdf.node[0]] = len(component_path[rootdf.node[0]])
                node_name[rootdf.node[0]] = self.lookup_with_node(root)['module'][0]
                entry_func[rootdf.node[0]] = True
                show_node[rootdf.node[0]] = True
                module[rootdf.node[0]] = group_path[rootdf.node[0]][-1]
                module_idx[rootdf.node[0]] = module_count

                print("entry function:", entry_func[rootdf.node[0]])
                print('Change name:', change_name[rootdf.node[0]])
                print("node path: ", root.callpath)                
                print("group path: ", group_path[rootdf.node[0]])
                print("component path: ", component_path[rootdf.node[0]])
                print("component level: ", component_level[rootdf.node[0]])
                print("Show node: ", show_node[rootdf.node[0]])
                print("name: ", node_name[rootdf.node[0]])
                print('Module: ', module[rootdf.node[0]])
                print("=================================")

            root = next(node_gen)

            try:
                while root.callpath != None:
                    root = next(node_gen)
                    s = self.lookup_with_name(root.callpath[-1])
                    parents = root.parents 
                    
                    for idx, parent in enumerate(parents):
                        t = self.lookup_with_name(parent.callpath[-1])
                    
                        if s.empty:
                            print("Not considering the Source function {0} [{1}]".format(parent, s['module']))
                        elif t.empty:
                            print("Not considering the Target function {0} [{1}]".format(root, t['path']))
                        elif not s.empty and not t.empty:
                            snode = s.node.tolist()[0]
                            tnode = t.node.tolist()[0]

                            spath = root.callpath
                            tpath = parent.callpath

                            tmodule = t[self.group_by].tolist()[0]

                            temp_group_path_results = self.create_group_path(spath)               
                            group_path[snode] = temp_group_path_results[0]
                            change_name[snode] = temp_group_path_results[1]
                            
                            component_path[snode] = self.create_component_path(spath, group_path[snode])
                            component_level[snode] = len(component_path[snode])
                            module[snode] = component_path[snode][0]
                            
                            if module[snode] not in module_id_map:
                                module_count += 1 
                                module_id_map[module[snode]] = module_count
                                module_idx[snode] = module_id_map[module[snode]]
                            else:
                                module_idx[snode] = module_id_map[module[snode]]

                            if component_level[snode] == 2:
                                entry_func[snode] = True
                                node_name[snode] = component_path[snode][0]
                                show_node[snode] = True
                            else:
                                entry_func[snode] = False
                                node_name[snode] = "Unknown(NA)"
                                show_node[snode] = False
                            
                    # print('Node', snode)        
                    # print("entry function:", entry_func[snode])
                    # print('Change name:', change_name[snode])
                    # print("node path: ", spath)                
                    # print("group path: ", group_path[snode])
                    # print("component path: ", component_path[snode])
                    # print("component level: ", component_level[snode])
                    # print("Show node: ", show_node[snode])
                    # print("name: ", node_name[snode])
                    # print('Module: ', module[snode])
                    # print("=================================")
                
            except StopIteration:
                pass
            finally:
                del root

        self.state.update_df('group_path', group_path)
        self.state.update_df('component_path', component_path)
        self.state.update_df('show_node', entry_func)
        self.state.update_df('vis_node_name', node_name)
        self.state.update_df('component_level', component_level)
        self.state.update_df('_'+self.group_by, module)
        self.state.update_df('change_name', change_name)
        self.state.update_df('mod_index', module_idx)   

        self.state.entry_funcs = self.entry_funcs
        self.state.other_funcs = self.other_funcs

In [58]:
groupby = groupBy(states['calc-pi'], 'module')

entry function: True
Change name: False
node path:  ('<program root>',)
group path:  ('libmonitor.so.0.0.0',)
component path:  ('libmonitor.so.0.0.0', '<program root>')
component level:  2
Show node:  True
name:  libmonitor.so.0.0.0
Module:  libmonitor.so.0.0.0


In [68]:
for idx, row in states['calc-pi'].df.iterrows():
    print(row['name'], row['component_path'])

<program root> ('libmonitor.so.0.0.0', '<program root>')
<program root> ('libmonitor.so.0.0.0', '<program root>')
<program root> ('libmonitor.so.0.0.0', '<program root>')
<program root> ('libmonitor.so.0.0.0', '<program root>')
main ('cpi', 'main')
main ('cpi', 'main')
main ('cpi', 'main')
main ('cpi', 'main')
62:MPI_Finalize ('libmonitor.so.0.0.0:62:MPI_Finalize', '62:MPI_Finalize')
62:MPI_Finalize ('libmonitor.so.0.0.0:62:MPI_Finalize', '62:MPI_Finalize')
62:MPI_Finalize ('libmonitor.so.0.0.0:62:MPI_Finalize', '62:MPI_Finalize')
62:MPI_Finalize ('libmonitor.so.0.0.0:62:MPI_Finalize', '62:MPI_Finalize')
PMPI_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize')
PMPI_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize')
PMPI_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize')
PMPI_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize')
294:MPID_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize', '294:MPID_Finalize')
294:MPID_Finalize ('libmpi.so.12.0.5', 'PMPI_Finalize', '294:MPID_Finalize')
294:MPID_Finalize ('libmpi

In [62]:
print(states['calc-pi'].entry_funcs)

{'Unknown(NA)': ['<unknown file>:0'], 'cpi': ['main'], 'libmonitor.so.0.0.0': [], 'libmpi.so.12.0.5': ['PMPI_Finalize'], 'libpsm_infinipath.so.1.14': ['36:<unknown procedure>']}


In [101]:
cct = CCT(state, 10)

1
[('<program root>', 'main', {'count': 1}), ('main', '62:MPI_Finalize', {'count': 1}), ('62:MPI_Finalize', 'PMPI_Finalize', {'count': 1}), ('PMPI_Finalize', '294:MPID_Finalize', {'count': 1}), ('294:MPID_Finalize', '162:MPIDI_CH3_Finalize', {'count': 1}), ('162:MPIDI_CH3_Finalize', '230:psm_dofinalize', {'count': 1}), ('230:psm_dofinalize', '36:<unknown procedure>', {'count': 1}), ('36:<unknown procedure>', '<unknown procedure>', {'count': 1}), ('<unknown procedure>', '<unknown procedure>', {'count': 9}), ('<unknown procedure>', '<unknown file>:0', {'count': 1})]
